In [1]:
import os
from dotenv import load_dotenv

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract second integer from first."""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b


def divide(a: int, b: int) -> float:
    """Divide first integer by second."""
    return a / b


In [4]:
tools = [add, subtract, multiply, divide]

In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
  model = 'gpt-4o-mini',
  )

llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [7]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages

class MessageState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

system_message = SystemMessage(
    content="You are a helpful assistant that can perform basic arithmetic operations."
)
def assistant(state:MessageState):
    return {"messages":[llm_with_tools.invoke([system_message] + state["messages"])]}

In [9]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import display, Image

builder = StateGraph(MessageState)

builder.add_node('assistant', assistant)
builder.add_node('tools', ToolNode(tools))

builder.add_edge(START, 'assistant')
builder.add_conditional_edges(
    'assistant', 
    #if the latest msg from the assistant is a tool call --> tool condition routes to the tools node and if the latest msg is not a tool call tools_condition routes to END
    tools_condition
    )

builder.add_edge('tools', 'assistant')

react_graph = builder.compile()
# display(Image(react_graph.get_graph().draw_mermaid_png()))


# mermaid_code = react_graph.get_graph().draw_mermaid()

# # Print the Mermaid code
# print(mermaid_code)

In [10]:
messages = [HumanMessage(content="What is 2 + 23 - 2 multiply by 13 and divide by 3?")]
messages = react_graph.invoke({'messages': messages})

In [11]:
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

What is 2 + 23 - 2 multiply by 13 and divide by 3?
================================== Ai Message ==================================
Tool Calls:
  add (call_Vw56R4y9E6J2co3wRB5ldlpV)
 Call ID: call_Vw56R4y9E6J2co3wRB5ldlpV
  Args:
    a: 2
    b: 23
================================= Tool Message =================================
Name: add

25
================================== Ai Message ==================================
Tool Calls:
  subtract (call_XXpbx7cgG2BseYZeez7k2EGO)
 Call ID: call_XXpbx7cgG2BseYZeez7k2EGO
  Args:
    a: 25
    b: 2
================================= Tool Message =================================
Name: subtract

23
================================== Ai Message ==================================
Tool Calls:
  multiply (call_SytWh5Cz0jzGxXQZGROBFkhE)
 Call ID: call_SytWh5Cz0jzGxXQZGROBFkhE
  Args:
    a: 23
    b: 13
================================= Tool Message ====================

In [50]:
question = [HumanMessage(content='What is 12+13?')]
response = react_graph.invoke({'messages': question})
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

What is 12+13?
================================== Ai Message ==================================
Tool Calls:
  add (call_LVEx33oAnRVRHWWr4BNciodK)
 Call ID: call_LVEx33oAnRVRHWWr4BNciodK
  Args:
    a: 12
    b: 13
================================= Tool Message =================================
Name: add

25
================================== Ai Message ==================================

12 + 13 equals 25.


In [51]:
question = [HumanMessage(content='Multiply that by 2')]
response = react_graph.invoke({'messages': question})
for m in response['messages']:
    m.pretty_print()  

================================ Human Message =================================

Multiply that by 2
================================== Ai Message ==================================

Please provide the number you would like to multiply by 2.


#### Here if we see the llm was not able to respond or make a tool call because it does not have a memory of previous messages

## Memory in Agents using MemorySaver

In [52]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

react_graph = builder.compile(checkpointer=memory)

In [53]:
config = {'configurable': {'thread_id': '1'}}

In [56]:
question = [HumanMessage(content='What is 12+13?')]
response = react_graph.invoke({'messages': question}, config)

In [57]:
for each_interaction in response['messages']:
    each_interaction.pretty_print()

================================ Human Message =================================

What is 12+13?
================================== Ai Message ==================================
Tool Calls:
  add (call_Yxbpmm7iUlYch8FGgakR9GZd)
 Call ID: call_Yxbpmm7iUlYch8FGgakR9GZd
  Args:
    a: 12
    b: 13
================================= Tool Message =================================
Name: add

25
================================== Ai Message ==================================

12 + 13 equals 25.


In [58]:
question = [HumanMessage(content='Now multiply that by 2')]
response = react_graph.invoke({'messages':question}, config)

In [59]:
for each_interaction in response['messages']:
    each_interaction.pretty_print()

================================ Human Message =================================

What is 12+13?
================================== Ai Message ==================================
Tool Calls:
  add (call_Yxbpmm7iUlYch8FGgakR9GZd)
 Call ID: call_Yxbpmm7iUlYch8FGgakR9GZd
  Args:
    a: 12
    b: 13
================================= Tool Message =================================
Name: add

25
================================== Ai Message ==================================

12 + 13 equals 25.
================================ Human Message =================================

Now multiply that by 2
================================== Ai Message ==================================
Tool Calls:
  multiply (call_VNyfLoMu0w1zxoa20rejQI8g)
 Call ID: call_VNyfLoMu0w1zxoa20rejQI8g
  Args:
    a: 25
    b: 2
================================= Tool Message =================================
Name: multiply

50
================================== Ai Message ==================================

25 multiplied by 2